In [1]:
# Import block
import pandas as pd
import numpy as np
import sklearn    # Comes with kNN method, may be useful

In [8]:
# Data loading block
# The MovieLens-20M dataset is split over 6 .csv files
# 1. genome-tags:    Associates each movie tag with a number
# 2. tags:           Each user and what tags they have assigned to which movies, with timestamp
# 3. genome-scores:  Each tag's relevance to every movie as a double 0-1
# 4. ratings:        20M ratings users gave to movies they watched (0.5-5, steps of 0.5)
# 5. movies:         Each movie's ID number and name, as well as genres 
# 6. links:          Each movie's entry numbers in IMDB and themovieDB (seemingly irrelevant?)

# Step 1: Convert all tags in 'tags' to their respective number to save space in the long run
genome_tags = pd.read_csv("genome-tags.csv") # get tags
user_tags = pd.read_csv("tags.csv", usecols=[0, 1, 2]) # get user-given tags, but not timestamp
genome_scores = pd.read_csv("genome-scores.csv")
ratings = pd.read_csv("ratings.csv", usecols=[0, 1, 2]) # get user-provided ratings
movies = pd.read_csv("movies.csv")
# links = pd.read_csv("links.csv")

In [12]:
print("Items in genome-tags:\t\t" + str(len(genome_tags.index)))
print("Items in user-tags:\t\t" + str(len(user_tags.index)))
print("Items in genome-scores:\t\t" + str(len(genome_scores.index)))
print("Items in ratings:\t\t" + str(len(ratings.index)))
print("Items in movies:\t\t" + str(len(movies.index)))

Items in genome-tags:		1128
Items in user-tags:		465564
Items in genome-scores:		11709768
Items in ratings:		20000263
Items in movies:		27278


In [10]:
# Create a user-movie matrix where rows=users and columns=movie ratings


# Notes: user tags are almost useless - many include their own and stray from the genome tags

# RATINGS appear important - other users' ratings on movies in the same GENRE would affect recs


1128

465548

38643

## "12/11/2019"